### Soumya Shrivastava
### Neural Network Hyper Parameter tuning

In [ ]:
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
# Importing Training and Test data
train=pd.read_csv('C:/Users/Soumya/Desktop/DL/digit-recognizer/train.csv')
test=pd.read_csv('C:/Users/Soumya/Desktop/DL/digit-recognizer/test.csv')
print('Training Data shape: ',train.shape)
print('Test Data shape: ',test.shape)

Training Data shape:  (42000, 785)
Test Data shape:  (28000, 784)


In [4]:
x=train.drop("label",axis=1)
y=np.array(train['label'])

In [6]:
# Convert Data to Tensor
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)

myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))

# Divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} , len of testSet {len(testSet)}")


# Load Data in memory
train_loader  = DataLoader(trainSet , batch_size=64 ,shuffle=True) 
test_loader  = DataLoader(testSet , batch_size=64 ,shuffle=True)

len of trainSet 33600 , len of testSet 8400


In [10]:
from torch import nn, optim
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64,32)
        self.fc5 = nn.Linear(32,10)

        # Dropout module with 0.1 drop probability
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))

        # No dropout at output
        x = F.log_softmax(self.fc5(x), dim=1)

        return x
        
model=Network()
print(model)
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr=0.3)

Network(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.1)
)


In [11]:
epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in train_loader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in test_loader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(test_loader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))

Epoch: 1/10..  Training Loss: 0.909..  Test Loss: 0.271..  Test Accuracy: 0.922
Epoch: 2/10..  Training Loss: 0.242..  Test Loss: 0.157..  Test Accuracy: 0.955
Epoch: 3/10..  Training Loss: 0.164..  Test Loss: 0.129..  Test Accuracy: 0.964
Epoch: 4/10..  Training Loss: 0.130..  Test Loss: 0.136..  Test Accuracy: 0.962
Epoch: 5/10..  Training Loss: 0.098..  Test Loss: 0.101..  Test Accuracy: 0.972
Epoch: 6/10..  Training Loss: 0.087..  Test Loss: 0.108..  Test Accuracy: 0.969
Epoch: 7/10..  Training Loss: 0.074..  Test Loss: 0.094..  Test Accuracy: 0.974
Epoch: 8/10..  Training Loss: 0.064..  Test Loss: 0.109..  Test Accuracy: 0.971
Epoch: 9/10..  Training Loss: 0.059..  Test Loss: 0.098..  Test Accuracy: 0.973
Epoch: 10/10..  Training Loss: 0.054..  Test Loss: 0.117..  Test Accuracy: 0.969
